# Fine-Tune a Causal Language Model for Dialogue Summarization

In this exercise, you will fine-tune a small Llama-like LLM called TinyLlama, for enhanced dialogue summarization. We will explore how to use the Huggingface TRL (Transformer Reinforcement Learning) library to help us to perform Supervised Finetuning (SFT).  We will explore the use of Parameter Efficient Fine-Tuning (PEFT) for efficient and fast finetuning, and evaluate the resulting model using ROUGE metrics.

The techniques you learnt will allow you to fine-tune other more powerful models such as LLama 3 for domain-specific application of LLM.

In [1]:
%%capture 
!pip install -q accelerate peft bitsandbytes transformers trl sentencepiece

In [2]:
import os 

os.environ['CUDA_VISIBLE_DEVICES']="0"

In [3]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from datasets import load_dataset
import torch

## Templating Instruction Data

To have the LLM follow instructions, we will need to prepare instruction data that follows a chat template. 

<img src="./chat_template.png" />

This chat template differentiates between what the LLM has generated and what the user has generated. May LLM chat models that are available on HuggingFace comes with built-in chat template that you can use. 

In [4]:
# This is the chat model of TinyLlama. We only load it because we want to use it's chat template to format our data
chat_model="meta-llama/Llama-3.2-1B-Instruct"

template_tokenizer = AutoTokenizer.from_pretrained(chat_model)

In [5]:
template_tokenizer.get_chat_template()

'{{- bos_token }}\n{%- if custom_tools is defined %}\n    {%- set tools = custom_tools %}\n{%- endif %}\n{%- if not tools_in_user_message is defined %}\n    {%- set tools_in_user_message = true %}\n{%- endif %}\n{%- if not date_string is defined %}\n    {%- if strftime_now is defined %}\n        {%- set date_string = strftime_now("%d %b %Y") %}\n    {%- else %}\n        {%- set date_string = "26 Jul 2024" %}\n    {%- endif %}\n{%- endif %}\n{%- if not tools is defined %}\n    {%- set tools = none %}\n{%- endif %}\n\n{#- This block extracts the system message, so we can slot it into the right place. #}\n{%- if messages[0][\'role\'] == \'system\' %}\n    {%- set system_message = messages[0][\'content\']|trim %}\n    {%- set messages = messages[1:] %}\n{%- else %}\n    {%- set system_message = "" %}\n{%- endif %}\n\n{#- System message #}\n{{- "<|start_header_id|>system<|end_header_id|>\\n\\n" }}\n{%- if tools is not none %}\n    {{- "Environment: ipython\\n" }}\n{%- endif %}\n{{- "Cutting

You can see that the template expects the prompt to include fields like role, content, and with content demarcated by `|user|`, `|assistant|` and `|system|`.

### Format the data according to chat template 

Let's download our data and format them according to the template given. We select a subset of 6000 samples to reduce training time. 


In [20]:
dataset_name = "knkarthick/dialogsum"
dataset_train = load_dataset(dataset_name, split='train[:3000]')
dataset_val = load_dataset(dataset_name, split='validation')
dataset_test = load_dataset(dataset_name, split='test')

In [21]:
dataset_train

Dataset({
    features: ['id', 'dialogue', 'summary', 'topic'],
    num_rows: 3000
})

Note that the completed prompt is put under 'text' field of the json. This is the default field that model will look for the text data.

In [22]:
def format_chat_template(row):
    user_prompt = (
        f"Summarize this dialog:\n{{dialog}}\n---\nSummary:\n"
    )
    user_prompt = user_prompt.format(dialog = row["dialogue"])
    row_json = [ {"role": "system", "content": "You are a helpful assistant" },
                {"role": "user", "content": user_prompt},
               {"role": "assistant", "content": row["summary"]}]

    prompt = template_tokenizer.apply_chat_template(row_json, tokenize=False)
    # print(prompt)
    return {"text": prompt}

In [23]:
dataset_train = dataset_train.map(format_chat_template, remove_columns=list(dataset_train.features))
dataset_val = dataset_val.map(format_chat_template, remove_columns=list(dataset_val.features))
dataset_test = dataset_test.map(format_chat_template, remove_columns=list(dataset_test.features))

Map:   0%|          | 0/3000 [00:00<?, ? examples/s]

Using the "text" column, we can explore these formatted prompts:

In [10]:
dataset_train[0]['text']

"<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 27 Oct 2024\n\nYou are a helpful assistant<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nSummarize this dialog:\n#Person1#: Hi, Mr. Smith. I'm Doctor Hawkins. Why are you here today?\n#Person2#: I found it would be a good idea to get a check-up.\n#Person1#: Yes, well, you haven't had one for 5 years. You should have one every year.\n#Person2#: I know. I figure as long as there is nothing wrong, why go see the doctor?\n#Person1#: Well, the best way to avoid serious illnesses is to find out about them early. So try to come at least once a year for your own good.\n#Person2#: Ok.\n#Person1#: Let me see here. Your eyes and ears look fine. Take a deep breath, please. Do you smoke, Mr. Smith?\n#Person2#: Yes.\n#Person1#: Smoking is the leading cause of lung cancer and heart disease, you know. You really should quit.\n#Person2#: I've tried hundreds of times, but I just can'

### Model Quantization

Now that we have our data, we can start loading in our model. This is where we apply the Q in QLoRA, namely quantization. We use the
bitsandbytes package to compress the pretrained model to a 4-bit representation.

In BitsAndBytesConfig, you can define the quantization scheme. We follow the steps used in the original QLoRA paper and load the model in 4-bit (load_in_4bit) with a normalized float representation (bnb_4bit_quant_type) and double quantization (bnb_4bit_use_double_quant):

In [26]:
import torch

from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

model_name = "meta-llama/Llama-3.2-1B"

# 4-bit quantization configuration - Q in QLoRA
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True, # Use 4-bit precision model loading
    bnb_4bit_quant_type="nf4", # Quantization type
    bnb_4bit_compute_dtype="float16", # Compute dtype
    bnb_4bit_use_double_quant=True, # Apply nested quantization
)

# Load the model to train on the GPU
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="cuda:0",
    # Leave this out for regular SFT
    quantization_config=bnb_config,
)

### Test the Model with Zero Shot Inferencing

Let's test the base model (non-instruction tuned model) with zero shot inferencing (i.e. ask it to summarize without giving any example. You can see that the model struggles to summarize the dialogue compared to the baseline summary, and it is just repeating the conversation.

In [12]:
eval_prompt = """
Summarize this dialog:
#Person1#: I have a problem with my cable.
#Person2#: What about it?
#Person1#: My cable has been out for the past week or so.
#Person2#: The cable is down right now. I am very sorry.
#Person1#: When will it be working again?
#Person2#: It should be back on in the next couple of days.
#Person1#: Do I still have to pay for the cable?
#Person2#: We're going to give you a credit while the cable is down.
#Person1#: So, I don't have to pay for it?
#Person2#: No, not until your cable comes back on.
#Person1#: Okay, thanks for everything.
#Person2#: You're welcome, and I apologize for the inconvenience.
---
Summary:
"""

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
model_input = tokenizer(eval_prompt, return_tensors="pt").to("cuda:0")

model.eval()
with torch.no_grad():   # no gradient update
    print(tokenizer.decode(model.generate(**model_input, max_new_tokens=200)[0], skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)



Summarize this dialog:
#Person1#: I have a problem with my cable.
#Person2#: What about it?
#Person1#: My cable has been out for the past week or so.
#Person2#: The cable is down right now. I am very sorry.
#Person1#: When will it be working again?
#Person2#: It should be back on in the next couple of days.
#Person1#: Do I still have to pay for the cable?
#Person2#: We're going to give you a credit while the cable is down.
#Person1#: So, I don't have to pay for it?
#Person2#: No, not until your cable comes back on.
#Person1#: Okay, thanks for everything.
#Person2#: You're welcome, and I apologize for the inconvenience.
---
Summary:
#Person1# has a problem with his cable.
#Person2# asks what's wrong.
#Person1# says his cable is down.
#Person2# says he's sorry.
#Person1# asks when it will be working again.
#Person2# says it should be back on in the next couple of days.
#Person1# asks if he still has to pay for the cable.
#Person2# says no, not until his cable comes back on.
#Person1# sa

### LoRA Configuration

We will be using LoRA to train our model. LoRA is supported in Hugging Face's PEFT library. 
Here are some explanation about the parameters used in the LoRA: 
- `r` - This is the rank of the compressed matrices. Increasing this value will also increase the sizes of compressed matrices leading to less compression and thereby improved representative power. Values typically range between 4 and 64.
- `lora_alpha` - Controls the amount of change that is added to the original weights. In essence, it balances the knowledge of the original model with that of the new task. A rule of thumb is to choose a value twice the size of r.
- `target_modules` - Controls which layers to target. The LoRA procedure can choose to ignore specific layers, like specific projection layers. This can speed up training but reduce performance and vice versa.

In [27]:
from peft import LoraConfig, prepare_model_for_kbit_training, get_peft_model

# Prepare LoRA Configuration
peft_config = LoraConfig(
    lora_alpha=32,  # LoRA Scaling
    lora_dropout=0.1,  # Dropout for LoRA Layers
    r=64,  # Rank
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=  # Layers to target
     ['k_proj', 'gate_proj', 'v_proj', 'up_proj', 'q_proj', 'o_proj', 'down_proj']
)

# prepare model for training
model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, peft_config)

### Training Configuration

Next we need to set our training configuration. Since we are going to use SFTTrainer, we can specify the training arguments in SFTConfig. 

Note that we set `fp16` to True for mixed-precision training. If you are using Ampere and newer GPU architecture, you can set bf16 to better accuracy and faster training.

In [28]:
from trl import SFTConfig

model.config.use_cache = False
model.config.pretraining_tp = 1

# Configure the tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

# where to write the checkpoint to
output_dir = "./results"

sft_config = SFTConfig(
    output_dir=output_dir,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=5,
    gradient_accumulation_steps=4,
    optim="paged_adamw_32bit",
    learning_rate=2e-4,
    lr_scheduler_type="cosine",
    # num_train_epochs=1,
    logging_steps=5,
    max_steps=30,
    bf16=True,
    # fp16=True
    gradient_checkpointing=True,
    resume_from_checkpoint=True,
    packing=True, 
    eval_packing=True,
    dataset_text_field="text",
    max_seq_length=1024,
    save_strategy = "steps",
    save_steps=5,
    eval_strategy='steps',
    eval_steps=5,
    run_name="llama3.2-summarize"
)

In [29]:
from trl import SFTTrainer

# Set supervised fine-tuning parameters
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset_train,
    eval_dataset=dataset_val,
    # dataset_text_field="text",
    tokenizer=tokenizer,
    # Leave this out for regular SFT
    peft_config=peft_config,
    args=sft_config
 )

# Train model
trainer.train()



max_steps is given, it will override any value given in num_train_epochs
/home/ubuntu/miniconda3/envs/llmenv/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:421: UserWarning: You passed `packing=True` to the SFTTrainer/SFTConfig, and you are training your model with `max_steps` strategy. The dataset will be iterated until the `max_steps` are reached.
  warnings.warn(
/home/ubuntu/miniconda3/envs/llmenv/lib/python3.10/site-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]


Step,Training Loss,Validation Loss
5,2.298400,2.098717
10,2.010700,1.904796
15,1.874900,1.831629


/home/ubuntu/miniconda3/envs/llmenv/lib/python3.10/site-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]
/home/ubuntu/miniconda3/envs/llmenv/lib/python3.10/site-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]
/home/ubuntu/miniconda3/envs/llmenv/lib/python3.10/site-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocas

KeyboardInterrupt: 

In [30]:
# Save QLoRA weights
trainer.model.save_pretrained("Llama-3.2-1B-Summarizer-QLoRA")

In [31]:
def get_max_context_length(model):
    
    conf = model.config
    max_length = None
    
    for length_setting in ["n_positions", "max_position_embeddings", "seq_length"]:
        max_length = getattr(model.config, length_setting, None)
        if max_length:
            print(f"Found max context lenth: {max_length} in {length_setting}")
            break
    if not max_length:
        max_length = 1024
        print(f"Using default max context length: {max_length}")
        
    return max_length

max_context_length = get_max_context_length(model)
print('Maximum Context length: ', max_context_length)

Found max context lenth: 131072 in max_position_embeddings
Maximum Context length:  131072


In [32]:
from peft import AutoPeftModelForCausalLM

model = AutoPeftModelForCausalLM.from_pretrained(
    "Llama-3.2-1B-Summarizer-QLoRA",
    low_cpu_mem_usage=True,
    device_map="auto",
)

# Merge LoRA and base model
merged_model = model.merge_and_unload()

In [33]:
eval_prompt = """<|user|>
Summarize this dialog:
A: Hi Tom, are you busy tomorrow’s afternoon?
B: I’m pretty sure I am. What’s up?
A: Can you go with me to the animal shelter?.
B: What do you want to do?
A: I want to get a puppy for my son.
B: That will make him so happy.
A: Yeah, we’ve discussed it many times. I think he’s ready now.
B: That’s good. Raising a dog is a tough issue. Like having a baby ;-)
A: I'll get him one of those little dogs.
B: One that won't grow up too big;-)
A: And eat too much;-))
B: Do you know which one he would like?
A: Oh, yes, I took him there last Monday. He showed me one that he really liked.
B: I bet you had to drag him away.
A: He wanted to take it home right away ;-).
B: I wonder what he'll name it.
A: He said he’d name it after his dead hamster – Lemmy  - he's  a great Motorhead fan :-)))
---
Summary:</s>
<|assistant|>
"""

from transformers import TextStreamer

model_input = tokenizer(eval_prompt, return_tensors="pt").to("cuda")

#Streaming support
streamer = TextStreamer(tokenizer)
merged_model.generate(**model_input, streamer=streamer, max_length=512)
# merged_model.eval()
# with torch.no_grad():
#     print(tokenizer.decode(merged_model.generate(**model_input)[0], skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


<|begin_of_text|><|user|>
Summarize this dialog:
A: Hi Tom, are you busy tomorrow’s afternoon?
B: I’m pretty sure I am. What’s up?
A: Can you go with me to the animal shelter?.
B: What do you want to do?
A: I want to get a puppy for my son.
B: That will make him so happy.
A: Yeah, we’ve discussed it many times. I think he’s ready now.
B: That’s good. Raising a dog is a tough issue. Like having a baby ;-)
A: I'll get him one of those little dogs.
B: One that won't grow up too big;-)
A: And eat too much;-))
B: Do you know which one he would like?
A: Oh, yes, I took him there last Monday. He showed me one that he really liked.
B: I bet you had to drag him away.
A: He wanted to take it home right away ;-).
B: I wonder what he'll name it.
A: He said he’d name it after his dead hamster – Lemmy  - he's  a great Motorhead fan :-)))
---
Summary:</s>
<|assistant|>
#user# wants to get a puppy for #user#'s son. #user# suggests getting a little dog that won't grow up too big. #user# also thinks #us

tensor([[128000,     27,     91,    882,     91,    397,   9370,   5730,    553,
            420,   7402,    512,     32,     25,  21694,   8529,     11,    527,
            499,  13326,  16986,    753,  13658,   5380,     33,     25,    358,
           4344,   5128,   2771,    358,   1097,     13,   3639,    753,    709,
           5380,     32,     25,   3053,    499,    733,    449,    757,    311,
            279,  10065,  23756,     30,    627,     33,     25,   3639,    656,
            499,   1390,    311,    656,   5380,     32,     25,    358,   1390,
            311,    636,    264,  42289,    369,    856,   4538,    627,     33,
             25,   3011,    690,   1304,   1461,    779,   6380,    627,     32,
             25,  22335,     11,    584,   4070,  14407,    433,   1690,   3115,
             13,    358,   1781,    568,    753,   5644,   1457,    627,     33,
             25,   3011,    753,   1695,     13,    432,  51226,    264,   5679,
            374,    264,  11

Good reference:

https://wandb.ai/capecape/alpaca_ft/reports/How-to-Fine-tune-an-LLM-Part-3-The-HuggingFace-Trainer--Vmlldzo1OTEyNjMy
